In [28]:
import pandas as pd
import os

input_data = '../model_collocated_10d_filled/'

import pickle 
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import json 


model = pickle.load(open("model_random_forest_sss_sst_only.pkl", 'rb')) 

date = '20221029'

profiles = pd.read_feather(f'{input_data}/{date}.feather')

profiles.dropna(inplace=True)
profiles = profiles[profiles['LATITUDE'] >= -60]
profiles = profiles[profiles['LATITUDE'] <= 60]
#predictors_df = profiles.loc[:, ['LATITUDE', 'SSS', 'SST', 'SSH', 'UO', 'VO', 'MLD']]
#predictors_df = profiles.loc[:, ['SSS', 'SST']]
profiles


,LATITUDE,LONGITUDE,SSS,SST,SSH,MLD,UO,VO,ASAL_height0.5057600140571594,ASAL_height1.5558552742004395,...,CTEMP_height370.6884765625,CTEMP_height411.7938537597656,CTEMP_height457.6256103515625,CTEMP_height508.639892578125,CTEMP_height565.2922973632812,CTEMP_height628.0260009765625,CTEMP_height697.2586669921875,CTEMP_height773.3682861328125,CTEMP_height856.678955078125,CTEMP_height947.4478759765625
173,-59.976030,-123.732770,34.092274,1.261145,-1.180604,97.227577,-0.089172,-0.070825,34.092274,34.114997,...,2.294959,2.311457,2.290257,2.253204,2.221728,2.194714,2.160876,2.116174,2.059694,1.991819
174,-59.966640,-123.962050,34.106602,1.052466,-1.192732,108.325905,-0.078449,-0.060081,34.106602,34.128959,...,2.318166,2.314976,2.280034,2.239005,2.208652,2.184300,2.151100,2.105245,2.045769,1.974036
175,-59.900000,-23.400000,33.427620,-1.679911,-1.680596,19.989191,0.118034,-0.102539,33.427620,33.480701,...,0.726086,0.721332,0.711978,0.695744,0.665742,0.613279,0.522351,0.392326,0.281925,0.193413
176,-59.868851,113.280760,33.960144,-0.015777,-1.345308,75.603508,0.202981,-0.047334,33.960144,34.006274,...,2.105650,2.089538,2.061661,2.024861,1.987475,1.944730,1.895561,1.839612,1.773770,1.694737
177,-59.794390,150.513720,34.106697,-0.738630,-1.491616,53.435001,-0.055491,0.040824,34.106697,34.129293,...,1.804118,1.770849,1.730291,1.681986,1.623107,1.553547,1.471565,1.377113,1.273990,1.169677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,59.687700,-25.700500,35.056660,10.486351,-0.663354,61.721302,0.103530,0.032236,35.056660,35.140642,...,7.615625,7.405357,7.121543,6.740560,6.315930,5.854272,5.383401,4.951322,4.548359,4.220731
3132,59.863100,-25.168600,35.059608,10.460088,-0.661058,61.395435,0.116530,-0.007806,35.059608,35.239318,...,7.539770,7.371650,7.125092,6.750260,6.318515,5.849996,5.359043,4.939241,4.510163,4.180294
3133,59.869352,-24.626175,35.064445,10.452206,-0.669988,61.310028,0.089939,-0.025071,35.064445,35.070715,...,7.525550,7.354779,7.101918,6.720387,6.278050,5.801604,5.308556,4.877820,4.463586,4.143732
3134,59.903000,-47.377000,34.769550,5.278093,-0.790181,65.161888,-0.442284,0.164434,34.769550,34.796203,...,4.912241,4.852536,4.769944,4.686605,4.621860,4.517319,4.451042,4.358847,4.220267,4.141397


In [29]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  
    return distance

# Coordinates to find the closest point to
target_points = [(0, -130), (30, -40), (-50,30), (-35,100)]

closest_points_df = pd.DataFrame(columns=profiles.columns)

for target_lat, target_lon in target_points:
    profiles['Distance_to_point'] = profiles.apply(lambda row: haversine(row['LATITUDE'], row['LONGITUDE'], target_lat, target_lon), axis=1)
    closest_indices = profiles[profiles['Distance_to_point'] == profiles['Distance_to_point'].min()].index
    closest_points_df = pd.concat([closest_points_df, profiles.loc[closest_indices]], ignore_index=True)

print("Closest points:")
print(closest_points_df)

predictors_df = closest_points_df.loc[:, ['SSS', 'SST']]
p = model.predict(predictors_df)

np.savetxt("random_forest_vertical_profs_sss_sst.csv", p, delimiter=",", fmt='%f', header="")

Closest points:
    LATITUDE   LONGITUDE        SSS        SST       SSH         MLD  \
0  -0.948000 -128.586100  34.993805  23.410891  0.239398   18.583752   
1  30.860380  -40.444973  37.127003  25.777161  0.037649   39.024582   
2 -50.088465   29.980923  33.804840   3.282859 -1.039059   31.153439   
3 -35.791000  102.329000  35.344795  13.543819  0.119469  124.829620   

         UO        VO  ASAL_height0.5057600140571594  \
0 -0.525332 -0.146803                      34.993805   
1  0.126703 -0.067378                      37.127003   
2  0.429212 -0.945588                      33.804840   
3 -0.080813 -0.011779                      35.344795   

   ASAL_height1.5558552742004395  ...  CTEMP_height411.7938537597656  \
0                      34.995957  ...                       8.395171   
1                      37.116767  ...                      14.248256   
2                      33.825539  ...                       2.360406   
3                      35.406839  ...                 

/tmp/ipykernel_1013158/344438302.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  closest_points_df = pd.concat([closest_points_df, profiles.loc[closest_indices]], ignore_index=True)
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


In [30]:
closest_points_df

,LATITUDE,LONGITUDE,SSS,SST,SSH,MLD,UO,VO,ASAL_height0.5057600140571594,ASAL_height1.5558552742004395,...,CTEMP_height411.7938537597656,CTEMP_height457.6256103515625,CTEMP_height508.639892578125,CTEMP_height565.2922973632812,CTEMP_height628.0260009765625,CTEMP_height697.2586669921875,CTEMP_height773.3682861328125,CTEMP_height856.678955078125,CTEMP_height947.4478759765625,Distance_to_point
0,-0.948000,-128.586100,34.993805,23.410891,0.239398,18.583752,-0.525332,-0.146803,34.993805,34.995957,...,8.395171,7.938750,7.350812,6.719417,6.075629,5.625635,5.133553,4.588237,4.140651,189.280901
1,30.860380,-40.444973,37.127003,25.777161,0.037649,39.024582,0.126703,-0.067378,37.127003,37.116767,...,14.248256,13.422090,12.541600,11.632955,10.682836,9.737321,8.852418,8.099487,7.417414,104.751043
2,-50.088465,29.980923,33.804840,3.282859,-1.039059,31.153439,0.429212,-0.945588,33.804840,33.825539,...,2.360406,2.354908,2.355656,2.359637,2.360023,2.353591,2.339736,2.316929,2.282123,9.930735
3,-35.791000,102.329000,35.344795,13.543819,0.119469,124.829620,-0.080813,-0.011779,35.344795,35.406839,...,10.016465,9.776489,9.470032,9.129573,8.480932,7.584918,6.583316,5.340209,4.325675,228.689318


In [31]:
# Drop the specified columns
columns_to_drop = ['LATITUDE', 'LONGITUDE', 'SSS', 'SST', 'SSH', 'MLD', 'UO', 'VO', 'Distance_to_point']
closest_points_df = closest_points_df.drop(columns=columns_to_drop)

# Convert the DataFrame to a NumPy array
numpy_array = closest_points_df.to_numpy()

np.savetxt("cmems_vertical_profiles.csv", numpy_array, delimiter=",", fmt='%f', header="")

In [25]:
p

array([[34.99429181, 35.03460692, 35.07460896, 35.10913722, 35.11746459,
        35.12088519, 35.12367063, 35.1259922 , 35.12667712, 35.12754542,
        35.12887777, 35.13091989, 35.13353618, 35.13775853, 35.14159861,
        35.14818276, 35.15865934, 35.16564904, 35.16865507, 35.171508  ,
        35.17597905, 35.18041559, 35.17707325, 35.16869032, 35.15988707,
        35.14729184, 35.13037543, 35.11207848, 35.09041656, 35.06381694,
        35.03274189, 34.99709283, 34.96026477, 34.91783889, 34.86621013,
        34.80605114, 34.74363175, 34.68506112, 34.63379052, 34.5950883 ,
        34.57224337, 34.56976322, 34.5881512 , 34.61933078, 34.6522751 ,
        34.68232977, 23.39086143, 23.32807088, 23.26545467, 23.20393234,
        23.15258134, 23.10483714, 23.05641764, 23.00723418, 22.94829894,
        22.8853532 , 22.81605581, 22.73129079, 22.62494212, 22.49555204,
        22.32057422, 22.1132089 , 21.8262883 , 21.46452198, 21.06468814,
        20.64746007, 20.22036119, 19.77839928, 19.3